In [74]:
import pandas as pd
import numpy as np
import implicit as im
from scipy.sparse import coo_matrix

In [96]:
df=pd.read_csv('dataset_1.csv')
df=df.drop(['色相','Unnamed: 0','品名','色调','款式流水号'],axis=1)
print(df.columns)
from sklearn.model_selection import train_test_split

X_train, X_test,= train_test_split(df, test_size = 0.2, random_state = 0)

Index(['客户ID', '衣服单价', '衣服ID', '颜色深度', '年份', '季节', '小类号码', '颜色式样', '颜色大类',
       '颜色名称', '大类', '中类', '品牌定义廓形', '领型', '袖长', '系列', '尺寸组', '标准价', '色温',
       '色明度', '色纯度', '对比度', '廓形', '宽松度', '长度', '款设图-大小复杂度', '款设图-规则性',
       '款设图-曲直', '材质-垂感', '整体曲直', '量感', '场合', '领型-形状', '领型-曲直', '领型-量感',
       '裙型裤型'],
      dtype='object')


In [97]:
def read_data(table):
    items_id=table['衣服ID']
    user_id=table['客户ID']
    table['客户ID'] = table['客户ID'].astype("category")#降低内存使用不会改变显示,将object对象映射为int
    table['衣服ID'] = table['衣服ID'].astype("category")
     
    itemID = []
    for i in items_id:
        if i not in itemID:
            itemID.append(i)
      
    
    #去重
    userset=set(user_id)
    print('衣服种类数:',len(itemID))
    print('用户数:',len(userset))
    x=numpy.ones((len(user_id)))#默认同一件衣服客户只买了一件
    user_item = coo_matrix((x.astype(np.float32),#构造rating矩阵
                       (table['衣服ID'].cat.codes.copy(),#Series.cat.codes 属性来返回 category 类型用来表示每个值的整型值
                        table['客户ID'].cat.codes.copy())))
    return table,user_item,user_id,itemID

In [98]:
data,user_item,user_id,items_id=read_data(X_train)

衣服种类数: 304
用户数: 143


D:\anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [99]:
from implicit.als import AlternatingLeastSquares
from implicit.approximate_als import (AnnoyAlternatingLeastSquares, FaissAlternatingLeastSquares,
                                      NMSLibAlternatingLeastSquares)
from implicit.bpr import BayesianPersonalizedRanking
from implicit.nearest_neighbours import (BM25Recommender, CosineRecommender,
                                         TFIDFRecommender, bm25_weight)
MODELS = {"als":  AlternatingLeastSquares,
          "nmslib_als": NMSLibAlternatingLeastSquares,
          "annoy_als": AnnoyAlternatingLeastSquares,
          "faiss_als": FaissAlternatingLeastSquares,
          "tfidf": TFIDFRecommender,
          "cosine": CosineRecommender,
          "bpr": BayesianPersonalizedRanking,
          "bm25": BM25Recommender}


def get_model(model_name):
    model_class = MODELS.get(model_name)
    if not model_class:
        raise ValueError("Unknown Model '%s'" % model_name)

    # some default params
    if issubclass(model_class, AlternatingLeastSquares):
        params = {'factors': 30, 'dtype': np.float32, 'use_gpu': False}
    elif model_name == "bm25":
        params = {'K1': 100, 'B': 0.5}
    elif model_name == "bpr":
        params = {'factors': 63, 'use_gpu': False}
    else:
        params = {}

    return model_class(**params)


In [100]:
import time
model = get_model('als')
user_item = user_item.tocsr()
start = time.time()
model.fit(user_item)
item_factors=model.item_factors
user_factors=model.user_factors

In [101]:
print(user_factors)

[[ 0.47653437 -0.18794447 -0.02533166 ... -0.13603494 -0.46018374
  -0.20432426]
 [-0.02515732 -0.06129502  0.06826571 ... -0.08216421  0.05450931
  -0.04135058]
 [-0.14481077  0.15658735 -0.00404981 ...  0.24742535  0.24666849
  -0.15126352]
 ...
 [ 0.32714915 -0.27568477  0.31904396 ...  0.7258384  -0.10301073
  -0.44749475]
 [ 0.5326484   0.31700242  0.41876262 ...  0.09785032  0.99020284
   0.1776728 ]
 [-0.11114027  0.03515346 -0.03932198 ... -0.00564214 -0.0379339
   0.01993871]]


In [102]:
user_factors[0].sum()

1.7993798

In [103]:
from sklearn import preprocessing
items_factors_normalized = preprocessing.normalize(item_factors, norm='l2')#L2正则化

In [104]:
item_factors

array([[ 0.12954001, -0.04235627,  0.02584365, ..., -0.01976464,
        -0.01140104, -0.08005229],
       [ 0.0086736 ,  0.00565025,  0.00503963, ..., -0.00625512,
        -0.00430229,  0.00916936],
       [-0.01826945,  0.05176275,  0.06092834, ...,  0.06521706,
        -0.09769947, -0.00374071],
       ...,
       [-0.0280849 ,  0.0116902 ,  0.05689881, ...,  0.00901342,
         0.01889084,  0.00097681],
       [-0.02204305, -0.01627204,  0.055082  , ...,  0.0223621 ,
        -0.01992983, -0.06905277],
       [ 0.02257465,  0.09849942,  0.00959947, ..., -0.01321794,
         0.11375605,  0.04643996]], dtype=float32)

In [105]:
item_features=pd.read_csv('item_feature.csv')
item_features.index=item_features['衣服ID']
item_features=item_features.drop(['衣服ID'],axis=1)
pd_item_factor=pd.DataFrame(items_factors_normalized,index=items_id)

In [106]:
itemID=item_features.index
new_item_feature=preprocessing.normalize(item_features, norm='l2')
item_features =pd.DataFrame(new_item_feature,index=itemID)
item_features=pd.merge(item_features,pd_item_factor, how='left',right_index=True,left_index=True)

In [107]:
def compute_consin(item):
    x=np.zeros((322,322))
    for i in range(322):
        A=item[i]
        A=A[:,None]
        for j in range(322):
            B=item[j]
            B=B[:,None].T
            num = np.dot(B,A) #若为行向量则 A * B.T           
            denom = np.linalg.norm(A) * np.linalg.norm(B)  
            cos = num / denom #余弦值 
            sim = 0.5 + 0.5 * cos #归一化
            x[i,j]=sim
    return x
where_are_nan = np.isnan(item_features)  
item_features[where_are_nan] = 0   
item_features=preprocessing.normalize(item_features, norm='l2')
xx=compute_consin(item_features)

In [108]:
distances=pd.DataFrame(xx,index=itemID,columns=itemID)
distances

衣服ID,JWWC11101000,JWWC31301215,JWXC60102100,JWWC31001100,JWWX11204000,JWXX20103126,JWXX60201100,JWXC20207100,JWXX50014026,JWXX11502220,...,JWXC80205100,JWXC31102200,JWXX50030100,JWXQ41205000,JWXC31106115,JWXQ31012100,JWXX30701126,JWXQ51006157,JWWC31204126,JWWC40511115
衣服ID,,,,,,,,,,,,,,,,,,,,,
JWWC11101000,1.000000,0.578611,0.575609,0.567525,0.639312,0.648056,0.542726,0.619777,0.618277,0.548198,...,0.550222,0.539020,0.547597,0.778410,0.532999,0.507150,0.535375,0.530507,0.507852,0.769921
JWWC31301215,0.578611,1.000000,0.517601,0.662922,0.550856,0.552668,0.576084,0.576062,0.535069,0.597247,...,0.611911,0.619866,0.605078,0.578503,0.654478,0.576764,0.604341,0.597567,0.677848,0.624997
JWXC60102100,0.575609,0.517601,1.000000,0.579738,0.561651,0.583008,0.611981,0.628264,0.551099,0.536682,...,0.601079,0.677897,0.688985,0.590755,0.515337,0.507100,0.577378,0.582643,0.501620,0.534702
JWWC31001100,0.567525,0.662922,0.579738,1.000000,0.526550,0.609835,0.563143,0.540862,0.629601,0.585561,...,0.589239,0.586097,0.594278,0.550142,0.698614,0.653132,0.615935,0.801087,0.600967,0.570795
JWWX11204000,0.639312,0.550856,0.561651,0.526550,1.000000,0.596109,0.568759,0.598084,0.570927,0.570855,...,0.532936,0.546210,0.552684,0.565425,0.536642,0.539112,0.537360,0.484652,0.523310,0.529144
JWXX20103126,0.648056,0.552668,0.583008,0.609835,0.596109,1.000000,0.523419,0.561476,0.582950,0.539467,...,0.536979,0.512373,0.557611,0.597989,0.746501,0.476833,0.644005,0.625501,0.557811,0.580513
JWXX60201100,0.542726,0.576084,0.611981,0.563143,0.568759,0.523419,1.000000,0.569709,0.584223,0.544386,...,0.529462,0.591803,0.633786,0.542529,0.556242,0.668400,0.599485,0.554061,0.571451,0.567410
JWXC20207100,0.619777,0.576062,0.628264,0.540862,0.598084,0.561476,0.569709,1.000000,0.521326,0.677065,...,0.615813,0.646604,0.584595,0.573423,0.525369,0.594353,0.584003,0.558724,0.587069,0.576159
JWXX50014026,0.618277,0.535069,0.551099,0.629601,0.570927,0.582950,0.584223,0.521326,1.000000,0.517047,...,0.546275,0.505978,0.593895,0.604902,0.563814,0.616524,0.632647,0.646796,0.548057,0.558017


In [ ]:
def make_recommention(distances,itemID):#
    

In [109]:
recommender={}#推荐列表 
start = time.time()
a=X_train.groupby('客户ID')
for user,grouped in a:
    x={}#存放推荐物品字典       
    for item in itemID:
        if item in grouped['衣服ID']:
            continue
        elif len(grouped['衣服ID'])==0:
            x[item]=0
        else:    
            x[item]=max(distances.loc[item,grouped['衣服ID']])
    recommender[user]=x
print("trained model '%s' in %0.2fs", time.time() - start)

trained model '%s' in %0.2fs 38.90188550949097


In [110]:
#排序
for d in recommender:
    a=recommender[d]
    recommender[d]=sorted(a.items(),key=lambda a:a[1],reverse=True)

In [111]:
b=X_test.groupby('客户ID')
sum=0
fenzi=0
for user,grouped in b:
    num=len(grouped['衣服ID'])
    sum+=num#分母
    if user not in list(recommender):
        user=483
    g=recommender[user]
    g=[i for (i,j) in g]
    for item in grouped['衣服ID']: 
        rank=(g.index(item)+1)/len(g)
        fenzi+=rank
rank_1=fenzi/sum
print(rank_1)

0.41083850931677013
